In [1]:
import pickle
import pandas as pd
import seaborn as sns

In [2]:
with open('TRANSPOSED-SIMP-SENS.pkl', 'rb') as file:
    data = pickle.load(file)

AttributeError: Can't get attribute 'TSTransformer' on <module '__main__'>